In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, merge, Dense, TimeDistributed, Dropout
from keras.layers import Bidirectional, concatenate, SpatialDropout1D
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from numpy import dot


import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import add

Using TensorFlow backend.


In [2]:
R1 = ["Name","Address","Contact","Sex","Guardian's-Name"]
R2 = ["name","Adres","Phone-No","Gender","Father's-Name"]

In [3]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")

data = data.fillna(method="ffill")
data.tail(5)

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [4]:
def case_conv(s) :
    k = str(s).lower()

    return k

data["Word"] = data["Word"].apply(case_conv)

wordwa = list(set(data["Word"].values))
words = []

for w in wordwa :
    w = str(w)
    w = w.lower()
    words.append(w)
    
print("Words[0:10] : ")
print(words[0:10])
n_words = len(words); 
n_words

chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print("Number of characters : " + str(n_chars))

char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

Words[0:10] : 
['encircled', 'faustin', 'teacher', 'though', 'rigorous', 'ecc', 'abayi', 'competent', 'go-betweens', '340']
Number of characters : 72


In [6]:
print(chars)

{'t', '?', '.', '\x85', 'k', '2', 'l', '9', 'e', 's', '\xa0', '+', '(', ',', '[', '6', 'n', '"', 'o', 'é', ':', 'ö', '\x97', ']', '_', ')', 'x', '\x93', '°', '/', 'w', '5', 'j', '@', '4', 'b', '-', 'i', '\x91', 'p', 'r', 'ë', '3', '`', '\x92', ';', 'ü', '7', 'u', '%', 'd', 'z', '\x96', 'y', 'h', 'v', '&', '8', "'", '~', 'c', 'f', 'm', '1', 'g', 'q', '#', '0', '\x94', 'a', '$', '!'}


In [7]:
max_word_length = 25
embedded_char_vector_length = 74 # Length of character dictionary
char_feature_output = 10

X_char = []
for word in words :
    l1 = []
    #if (len(word) > max_word_length) :
    #    print(len(word))
    for i in range(max_word_length):
        try:
            l1.append(char2idx.get(word[i]))
        except:
            l1.append(char2idx.get("PAD"))    
    
    X_char.append(l1)

In [8]:
char_input = Input(shape=(max_word_length,), dtype='float32', name='char_input')
char_input1 = Embedding(embedded_char_vector_length, 16, input_length = max_word_length)(char_input)
char_input2 = Dropout(0.2)(char_input1)
'''lstm_out_forward = LSTM(char_feature_output, dropout=0.2, recurrent_dropout=0.2)(char_input2)
lstm_out_backward = LSTM(char_feature_output, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(char_input2)'''

merged = (Bidirectional(LSTM(units=5, return_sequences=False,trainable = False,
                                recurrent_dropout=0.5)))(char_input2)
#merged = merge([lstm_out_forward, lstm_out_backward], mode='concat', concat_axis=1)

#merged = add([lstm_out_forward, lstm_out_backward])
#model = Model(input=[char_input], output=[merged])
model = Model(char_input, merged)


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_input (InputLayer)      (None, 25)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 16)            1184      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 16)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10)                880       
Total params: 2,064
Trainable params: 2,064
Non-trainable params: 0
_________________________________________________________________


In [9]:
X1 = np.array(X_char[0:5])
print(X1)
train_char_vectors = model.predict(X1)


print("\n")
print(train_char_vectors.shape)
print("\n")
print(train_char_vectors)

[[10 18 62 39 42 62  8 10 52  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [63 71 50 11  2 39 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [ 2 10 71 62 56 10 42  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [ 2 56 20 50 66 56  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [42 39 66 20 42 20 50 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]]


(5, 10)


[[-0.01278956 -0.0260345   0.01869805 -0.00473439  0.04407868 -0.01404607
   0.00533008 -0.00632584  0.01047952 -0.02199801]
 [-0.01270991 -0.02606502  0.01834428 -0.00536459  0.04831399  0.00079215
   0.0051955   0.00110492 -0.00562118 -0.01084856]
 [-0.01338342 -0.02631043  0.01894302 -0.00462216  0.04605633 -0.00305164
   0.00850239 -0.00663072  0.0001973  -0.01376939]
 [-0.01313376 -0.02622901  0.01858731 -0.00513753  0.04865635  0.00441031
   0.01842081  0.00098834  0.00745362 -0.02542891]
 [-0.01318545 -0.02621371  0.01866391 -0.00503973  0.048139    0.01869542
   0.00124471

In [10]:
def pred_new_num_enc(s) :
    
    pred = s

    l = []
    ip = []
    k=0

    pred = pred.lower()
    print(pred)
    for i in range(len(pred)):
        l.append(char2idx.get(pred[i]))
    #print(l)
    #print(char2idx.get("PAD"))        


    length = len(l)
    #print(length)

    if length < max_word_length :
        k = max_word_length - length
        #print(k)

        for i in range (0,k) :
            l.append(char2idx.get("PAD"))    

    else :
        l = l[:max_word_length]
    #ip.append(l)

    l = np.array([l])
    #print(l)
    
    return l

In [11]:
def pred_new_vec(l) :
    train_char_vectors = model.predict(l)

    #print(train_char_vectors)
    
    return train_char_vectors

In [12]:
def form_list(R) :
    lR = []
    for r in R :
        a = pred_new_num_enc(r)
        #print(a)
        #print("\n")
        a = pred_new_vec(a)
        #print(a)
        #print("\n")    
        lR.append(a)
        #print(lR)
        
    lR = np.array(lR)
    #print(lR.shape)

    #np.reshape(lR1,(5,10))
    lR = lR[:,0,:]

    #print(lR1)
    #print("\n")
    
    print(lR.shape)
    print("\n")
    #print(lR)
    print(lR.shape)
    lR = lR.flatten()

    print(lR.shape)
    
    lR = list(lR)
    
    print(lR)
    print(len(lR))
    
    return lR

In [13]:
lR1 = form_list(R1)

name
address
contact
sex
guardian's-name
(5, 10)


(5, 10)
(50,)
[-0.013021096, -0.026189914, 0.018591857, -0.0050703157, 0.047639143, 0.015264592, 0.0031732074, 0.0065551354, 0.013116316, -0.019845117, -0.013105687, -0.026211847, 0.018798025, -0.00474335, 0.045635715, 0.02188702, 0.0045319214, -0.003092831, -0.0051387292, 0.024936318, -0.0127765685, -0.02609994, 0.018479738, -0.005143124, 0.047025107, -0.017181186, 0.019287255, -0.0110048605, 0.0017949457, -0.038013615, -0.013210144, -0.026249709, 0.018665785, -0.0050337045, 0.04825426, 0.039060928, 0.0046477765, -0.022801433, 0.023475593, -0.018840145, -0.012396035, -0.025010722, 0.018533623, -0.004664186, 0.04156445, 0.009309375, -0.010275597, 0.00042433196, 0.007938867, 0.0032581598]
50


In [14]:
lR2 = form_list(R2)

name
adres
phone-no
gender
father's-name
(5, 10)


(5, 10)
(50,)
[-0.013021096, -0.026189914, 0.018591857, -0.0050703157, 0.047639143, 0.015264592, 0.0031732074, 0.0065551354, 0.013116316, -0.019845117, -0.013161755, -0.026246034, 0.018759407, -0.0048410846, 0.046624597, 0.028205957, 0.00074135716, -0.004140925, 0.00048044918, 0.016458778, -0.012759748, -0.026080573, 0.018409828, -0.005279363, 0.047905996, -0.011589454, 0.013057915, 0.0038509131, 0.0065379683, -0.020734012, -0.013213311, -0.02625097, 0.018772546, -0.0048431805, 0.046873413, 0.0123586105, 0.0008097728, -0.008544381, 0.021937018, -0.015701042, -0.013045046, -0.025757855, 0.018995794, -0.004300287, 0.04196324, -0.01138396, 0.0055164737, 0.0016156811, -0.017188268, 0.00516878]
50


In [15]:
inputs_records = lR1 + lR2

print(inputs_records)

print("\n")
print(len(inputs_records))

[-0.013021096, -0.026189914, 0.018591857, -0.0050703157, 0.047639143, 0.015264592, 0.0031732074, 0.0065551354, 0.013116316, -0.019845117, -0.013105687, -0.026211847, 0.018798025, -0.00474335, 0.045635715, 0.02188702, 0.0045319214, -0.003092831, -0.0051387292, 0.024936318, -0.0127765685, -0.02609994, 0.018479738, -0.005143124, 0.047025107, -0.017181186, 0.019287255, -0.0110048605, 0.0017949457, -0.038013615, -0.013210144, -0.026249709, 0.018665785, -0.0050337045, 0.04825426, 0.039060928, 0.0046477765, -0.022801433, 0.023475593, -0.018840145, -0.012396035, -0.025010722, 0.018533623, -0.004664186, 0.04156445, 0.009309375, -0.010275597, 0.00042433196, 0.007938867, 0.0032581598, -0.013021096, -0.026189914, 0.018591857, -0.0050703157, 0.047639143, 0.015264592, 0.0031732074, 0.0065551354, 0.013116316, -0.019845117, -0.013161755, -0.026246034, 0.018759407, -0.0048410846, 0.046624597, 0.028205957, 0.00074135716, -0.004140925, 0.00048044918, 0.016458778, -0.012759748, -0.026080573, 0.018409828, 

In [16]:
'''arr = []

for rec in inputs_records :
    arr.append(rec)
    
print(arr)'''

'arr = []\n\nfor rec in inputs_records :\n    arr.append(rec)\n    \nprint(arr)'

In [17]:
inputs_records = np.array([inputs_records])
print(inputs_records.shape)

model_mlp = Sequential()
model_mlp.add(Dense(50, input_shape = (100,), activation='relu'))
model_mlp.add(Dense(1, activation='sigmoid'))

model_mlp.summary()

(1, 100)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [18]:
prediction = model_mlp.predict(inputs_records)
print(prediction)

[[0.49723434]]


In [19]:
print(round(prediction[0,0]))

0.0


NameError: name 'model_mlp' is not defined